In [4]:
import sys
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

sys.path.append('../Scripts')
from Data_Processing import DataProcessing

from tensorflow import keras
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras import backend as K
from datetime import datetime
from sklearn.preprocessing import PowerTransformer

import joblib

In [5]:
ColumnTransformer = joblib.load('../Models/Column_Transformer.pkl')
PowerTransformer = joblib.load('../Models/Power_Transformer.pkl')

In [6]:
df = DataProcessing('../Data/test.csv')

X = df.drop(columns=['Lap_Time'])

num_columns = X.select_dtypes(include='number').columns

In [7]:
X[num_columns] = PowerTransformer.transform(X[num_columns])
trans_X = ColumnTransformer.transform(X)

trans_X = trans_X.toarray()

In [8]:
def root_mean_squared_log_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(K.log(1+y_pred) - K.log(1+y_true))))

In [9]:
model = load_model('../Models/NN_model.h5', custom_objects={'root_mean_squared_log_error': root_mean_squared_log_error})

In [10]:
y_predicted = model.predict(trans_X)

In [11]:
submission = pd.DataFrame(y_predicted, columns=['LAP_TIME'])

In [12]:
today = datetime.today().strftime('%m-%d-%y %H-%M')
submission.to_csv(f'../Submissions/Dare_In_Reality {today}.csv', index=False)

In [13]:
submission

,LAP_TIME
0,79.311333
1,75.954353
2,74.219032
3,82.666389
4,79.264984
...,...
415,76.305672
416,67.653122
417,76.098839
418,68.266449
